# Yelp API - Lab

The previously deployed lab on working around building a GIS with Yelp API and Folium can be found [here](https://github.com/learn-co-curriculum/dsc-2-15-10-yelp-api-gis-lab/tree/a56358c2d0c2daf569a5f50937c4c27463aadb1a) (not relevant for new students).


## Introduction 

Now that we've seen how the Yelp API works, and some basic Folium visualizations its time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [6]:
#Your code here
import requests
api='aduJke_wFpzFfhQds6klmakwCD1XDup3eAHPgCOVRklbIYvSpIiRn3P4T-6tIkXuzCWwdo2dSVS0LYWzTKt_UTk4BTKemdeFzZVqF0RW0xUqhQgRQoveZBeokEVoXHYx'
r=requests.get('https://api.yelp.com/v3/businesses/search',headers={'Authorization':'Bearer '+api},params={'term':'sushi','location':'flatbush+brooklyn','limit':50,'offset':51})
r.status_code

200

In [27]:
r.json()['total']

224

In [28]:
len(r.json()['businesses'])

50

In [8]:
import pandas as pd
dfb=pd.DataFrame.from_dict(r.json()['businesses'])
dfb.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,sake-sushi-brooklyn,"[{'alias': 'japanese', 'title': 'Japanese'}, {...","{'latitude': 40.643961, 'longitude': -73.97669}",(718) 851-5399,1255.024287,CKBku3Flj5Nh8VA6ZH-q2w,https://s3-media1.fl.yelpcdn.com/bphoto/txNXdP...,False,"{'address1': '328 Church Ave', 'address2': '',...",Sake Sushi,+17188515399,$$,3.5,164,"[pickup, delivery]",https://www.yelp.com/biz/sake-sushi-brooklyn?a...
1,kumo-sushi-brooklyn-5,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...","{'latitude': 40.6406909, 'longitude': -73.964964}",(718) 282-8288,293.073705,jHn3mUjbe9caYZNe1z9YUw,https://s3-media1.fl.yelpcdn.com/bphoto/RfNjMI...,False,"{'address1': '1406 Cortelyou Rd', 'address2': ...",Kumo Sushi,+17182828288,$$,3.0,134,"[pickup, delivery]",https://www.yelp.com/biz/kumo-sushi-brooklyn-5...
2,asia-harbor-kensington-2,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...","{'latitude': 40.643948, 'longitude': -73.9763473}",(718) 871-0635,1229.566883,QGldR7s9_qV_35iGtkcNyA,https://s3-media3.fl.yelpcdn.com/bphoto/t5EI4U...,False,"{'address1': '402 Church Ave', 'address2': '',...",Asia Harbor,+17188710635,$$,3.5,73,"[pickup, delivery]",https://www.yelp.com/biz/asia-harbor-kensingto...
3,sushi-lin-brooklyn-2,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...","{'latitude': 40.677786, 'longitude': -73.972591}",(347) 627-6027,4068.085640,AFt1Qcec4_JNr6PWpkRYyw,https://s3-media1.fl.yelpcdn.com/bphoto/2zPxmK...,False,"{'address1': '335 Flatbush Ave', 'address2': '...",Sushi Lin,+13476276027,$$$,5.0,191,[],https://www.yelp.com/biz/sushi-lin-brooklyn-2?...
4,silver-rice-brooklyn-2,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...","{'latitude': 40.6597410822446, 'longitude': -7...",(347) 789-9875,1967.250445,1yvEVWnJfodsReGHc8DuVQ,https://s3-media2.fl.yelpcdn.com/bphoto/ECDHTT...,False,"{'address1': '575A Flatbush Ave', 'address2': ...",Silver Rice,+13477899875,$$,4.5,73,"[pickup, delivery]",https://www.yelp.com/biz/silver-rice-brooklyn-...


In [31]:
dfc=pd.concat([dfa,dfb])

In [33]:
dfa.shape

(50, 16)

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.josn().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [9]:
# Your code here; use a function or loop to retrieve all the results from your original request
r=requests.get('https://api.yelp.com/v3/businesses/search',headers={'Authorization':'Bearer '+api},params={'term':'sushi','location':'flatbush+brooklyn','limit':50,'offset':0})
df=pd.DataFrame.from_dict(r.json()['businesses'])
num_responses=r.json()['total']
offset=50
while offset <num_responses:
    r=requests.get('https://api.yelp.com/v3/businesses/search',headers={'Authorization':'Bearer '+api},params={'term':'sushi','location':'flatbush+brooklyn','limit':50,'offset':offset})
    df_temp=pd.DataFrame.from_dict(r.json()['businesses'])
    df=pd.concat([df,df_temp])
    offset+=50
df.shape

(224, 16)

## Exploratory Analysis

Take the restaurants from the previous question and do an intial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [35]:
#Your code here
df.columns

Index(['alias', 'categories', 'coordinates', 'display_phone', 'distance', 'id',
       'image_url', 'is_closed', 'location', 'name', 'phone', 'price',
       'rating', 'review_count', 'transactions', 'url'],
      dtype='object')

In [37]:
df.loc[df.rating==df.rating.max()]

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
3,sushi-lin-brooklyn-2,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...","{'latitude': 40.677786, 'longitude': -73.972591}",(347) 627-6027,4068.085640,AFt1Qcec4_JNr6PWpkRYyw,https://s3-media1.fl.yelpcdn.com/bphoto/2zPxmK...,False,"{'address1': '335 Flatbush Ave', 'address2': '...",Sushi Lin,+13476276027,$$$,5.0,191,[],https://www.yelp.com/biz/sushi-lin-brooklyn-2?...
34,poke-bowl-brooklyn-17,"[{'alias': 'hawaiian', 'title': 'Hawaiian'}]","{'latitude': 40.67636, 'longitude': -73.99908}",(718) 875-5998,4930.695551,3s0uwi3rECdbX64ZEC_fuQ,https://s3-media2.fl.yelpcdn.com/bphoto/36GOE2...,False,"{'address1': '512 Court St', 'address2': None,...",Poke Bowl,+17188755998,NaN,5.0,5,"[pickup, delivery]",https://www.yelp.com/biz/poke-bowl-brooklyn-17...
40,harajuku-sushi-and-crepe-new-york,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...","{'latitude': 40.7491302490234, 'longitude': -7...",(626) 789-4134,12797.730917,jva4AMX7zRkNsY3DKX9LlQ,https://s3-media4.fl.yelpcdn.com/bphoto/6vsrkj...,False,"{'address1': None, 'address2': None, 'address3...",Harajuku Sushi and Crepe,+16267894134,$,5.0,12,[],https://www.yelp.com/biz/harajuku-sushi-and-cr...
45,qius-house-brooklyn,"[{'alias': 'japanese', 'title': 'Japanese'}, {...","{'latitude': 40.61354, 'longitude': -73.98037}",(718) 234-0800,3534.717699,G6fxpE4WgMAKJZgooRAeMg,https://s3-media3.fl.yelpcdn.com/bphoto/O-IRLK...,False,"{'address1': '2229 65th St', 'address2': '', '...",Qiu's House,+17182340800,NaN,5.0,4,"[pickup, delivery]",https://www.yelp.com/biz/qius-house-brooklyn?a...
15,freunds-fish-market-brooklyn,"[{'alias': 'seafoodmarkets', 'title': 'Seafood...","{'latitude': 40.63549, 'longitude': -73.98405}",(718) 438-3773,2002.751994,0niXaomLyYYcM-1ykDvKGA,https://s3-media2.fl.yelpcdn.com/bphoto/34kzul...,False,"{'address1': '4301 15th Ave', 'address2': '', ...",Freund's Fish Market,+17184383773,$$,5.0,1,[],https://www.yelp.com/biz/freunds-fish-market-b...
3,tacos-el-gavilan-brooklyn,"[{'alias': 'foodtrucks', 'title': 'Food Trucks'}]","{'latitude': 40.6326477831296, 'longitude': -7...",(347) 488-8316,1683.620205,qldNlBZWuPeGTcZ4h27qrw,https://s3-media4.fl.yelpcdn.com/bphoto/fb69al...,False,"{'address1': '', 'address2': '', 'address3': '...",Tacos El Gavilan,+13474888316,NaN,5.0,4,[],https://www.yelp.com/biz/tacos-el-gavilan-broo...


In [10]:
df.iloc[0].location

{'address1': '328 Church Ave',
 'address2': '',
 'address3': '',
 'city': 'Brooklyn',
 'zip_code': '11218',
 'country': 'US',
 'state': 'NY',
 'display_address': ['328 Church Ave', 'Brooklyn, NY 11218']}

In [4]:
df.describe()

NameError: name 'df' is not defined

In [69]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.boxplot(df.price.astype('category'),df.rating, showfliers=False)
# plt.ylim(0,1000)
plt.show()

ValueError: 
$$
^
Expected end of text (at char 0), (line:1, col:1)

<Figure size 432x288 with 1 Axes>

In [83]:
df.coordinates.head()

0      {'latitude': 40.643961, 'longitude': -73.97669}
1    {'latitude': 40.6406909, 'longitude': -73.964964}
2    {'latitude': 40.643948, 'longitude': -73.9763473}
3     {'latitude': 40.677786, 'longitude': -73.972591}
4    {'latitude': 40.6597410822446, 'longitude': -7...
Name: coordinates, dtype: object

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [11]:
import folium
map2 = folium.Map([40.65,-73.95],zoom_start=14)
map2
for i in range(0,len(df)):
    coordinates=df.iloc[i]['coordinates']
    lat=coordinates['latitude']
    long=coordinates['longitude']
    address= dfb.iloc[i]['location']
    popup_text = "{} \n\nAddress:{}, \n\nRating: {}, \n\nPrice".format(dfb.iloc[i]['name'],address['display_address'],dfb.iloc[i]['rating'],dfb.iloc[i]['price'])
    popup = folium.Popup(popup_text, parse_html=True)
    marker=folium.Marker(location=[lat,long],popup=popup)
    marker.add_to(map2)
map2

ModuleNotFoundError: No module named 'folium'

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!